In [ ]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
  from google.colab import drive
  drive.mount('/content/drive')

  # Google Drive
  dir_base = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/data/"
  # raiz
  dir_base_root = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/"
  import sys
  # caution: path[0] is reserved for script path (or '' in REPL)
  sys.path.append(dir_base_root)

  # Insalar depedências


else:
  # Diretorio da base dos dados
  # Local PC
  dir_base = "data/"


Mounted at /content/drive


# Feature Selection

In [ ]:
# Importando as bibliotecas utilizadas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression


import pod_academy_functions as pod

-----------------------------------------------------
------------ PoD Academy Functions ------------------
Só Alegria!
Você está usando o pacote de funções da PoD Academy
Turma: 2023
Autor: PoD Academy
Professor: Bruno Jardim
Versão: 1.0.0
Data: 08/10/2023
-----------------------------------------------------


In [ ]:
# Carrega os dados para processo de feature selection
abt_00 = pd.read_csv(dir_base + 'abt_train.csv')

In [ ]:
abt_00.head()

,Unnamed: 0,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1,id,target
0,510263,0.553925,-0.543007,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1274939,0
1,566227,-0.957891,-0.543007,0.211296,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1415654,0
2,154123,-0.453952,0.961267,-0.898992,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,385376,1
3,500653,1.561802,-0.543007,-0.158800,-0.311952,-0.805988,1.703539,-0.443149,-0.47767,-0.019446,...,-0.63978,7.153559,-3.639785,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,1251174,0
4,362160,-0.957891,-0.543007,-1.269089,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,...,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,904785,0


In [ ]:
abt_00.drop(axis=1, columns=['Unnamed: 0', 'id'], inplace=True)
abt_00.shape

(357127, 61)

In [ ]:
abt_00.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,...,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1,target
0,0.553925,-0.543007,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-0.47767,-0.019446,-0.041262,...,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,0
1,-0.957891,-0.543007,0.211296,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,...,-0.900937,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,0
2,-0.453952,0.961267,-0.898992,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,...,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1
3,1.561802,-0.543007,-0.158800,-0.311952,-0.805988,1.703539,-0.443149,-0.47767,-0.019446,-0.041262,...,-0.900937,-0.63978,7.153559,-3.639785,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,0
4,-0.957891,-0.543007,-1.269089,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,...,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,0


In [ ]:
X = abt_00.drop(axis=1, columns='target')
y = abt_00.target

# Treinar o modelo com algoritmo LogisticRegression (foco aqui não é o modelo, e sim seleção de variáveis)
lr = LogisticRegression() # escolha aqui o modelo desejado. Podendo ser feito apenas com um modelo.

lr.fit(X,y)

LogisticRegression()

In [ ]:
#Seleção de features pelo RFE

sel_rfe = RFE(lr, n_features_to_select=15)

sel_rfe.fit(X, y)

print('Número de features selecionadas: ', sel_rfe.n_features_)

Número de features selecionadas:  15


In [ ]:
# lista de variaveis selecionadas
selected_features = X.columns[sel_rfe.support_].to_list()
selected_features

['ps_ind_03',
 'ps_ind_05_cat',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_15',
 'ps_ind_17_bin',
 'ps_reg_01',
 'ps_reg_03',
 'ps_car_09_cat',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15',
 'ps_car_05_cat_0.5261814898236643',
 'ps_car_07_cat_1.0']

In [ ]:
sel_rfe.ranking_

array([10, 13,  1,  1,  1,  1,  1,  2, 34, 39, 30, 26, 37,  1,  6,  1,  7,
        1,  5,  1, 11, 15, 17,  1, 38, 16,  3,  1,  1, 42,  1, 14, 31, 20,
       44, 29, 23, 24, 36, 43, 45, 32, 22, 35, 28, 19, 41, 40, 27, 12, 33,
        1, 21, 46,  1,  4,  8, 25, 18,  9])

In [ ]:
np.absolute(sel_rfe.estimator_.coef_)

array([[0.0385009 , 0.14419236, 0.02541212, 0.09920079, 0.0788858 ,
        0.10007175, 0.13026671, 0.0998736 , 0.06485487, 0.05609586,
        0.05191542, 0.07069684, 0.08026409, 0.0576201 , 0.08964609]])

In [ ]:
# Visualizar  as features selecionadas
# for i in range(X.shape[1]):
#   col_name = X.columns[i]
#   sel = sel_rfe.support_[i]
#   if sel == True:
#     print(col_name)

In [ ]:
# Salvar a lista de variaveis em um arquivo .pkl
import pickle
with open(dir_base + 'prd_list_features_rfe.pkl', 'wb') as f:
    pickle.dump(selected_features,f)

In [ ]:
# Carregar  a lista
with open(dir_base + 'prd_list_features_rfe.pkl', 'rb') as f:
    loaded_list = pickle.load(f)
print(loaded_list)

['ps_ind_03', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_15', 'ps_ind_17_bin', 'ps_reg_01', 'ps_reg_03', 'ps_car_09_cat', 'ps_car_12', 'ps_car_13', 'ps_car_15', 'ps_car_05_cat_0.5261814898236643', 'ps_car_07_cat_1.0']


In [ ]:
# Cria a lista de variáveis com o target
list_features = loaded_list + ['target']

# Cria um novo Dataframe apenas com as variáveis selecionadas
abt_01 = abt_00[list_features]

# Visualização
abt_01.head()

,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_15,ps_ind_17_bin,ps_reg_01,ps_reg_03,ps_car_09_cat,ps_car_12,ps_car_13,ps_car_15,ps_car_05_cat_0.5261814898236643,ps_car_07_cat_1.0,target
0,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-1.777632,2.697660,-1.083208,0.281831,0.685453,-0.096613,-0.616369,-0.326028,1.109956,0.274741,0
1,0.211296,-0.311952,1.240714,-0.587013,-0.443149,1.042904,-0.370692,-0.039750,-0.684878,-1.362990,1.153130,-1.003274,-2.828849,-0.900937,0.274741,0
2,-0.898992,-0.311952,1.240714,-0.587013,-0.443149,0.196743,2.697660,1.003708,-1.029515,0.685453,-1.092421,-0.508637,0.130960,1.109956,0.274741,1
3,-0.158800,-0.311952,-0.805988,1.703539,-0.443149,1.042904,-0.370692,1.003708,-0.702581,0.685453,1.158897,4.284008,0.544108,-0.900937,-3.639785,0
4,-1.269089,-0.311952,1.240714,-0.587013,-0.443149,-1.777632,-0.370692,1.003708,1.029377,0.685453,0.347414,-0.116975,0.342236,1.109956,0.274741,0


In [ ]:
# Carrega os dados de teste
abt_test = pd.read_csv(dir_base + 'abt_test.csv')
# Remove a coluna desnecessária
abt_test.drop(axis=1, columns=['Unnamed: 0', 'id'], inplace=True)
abt_test.shape

(238085, 61)

In [ ]:
# Cria a lista de variáveis com o target
list_features = loaded_list + ['target']
# Cria um novo Dataframe apenas com as variáveis selecionadas
abt_01_test = abt_test[list_features]
# Visualização
abt_01_test.head()

,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_15,ps_ind_17_bin,ps_reg_01,ps_reg_03,ps_car_09_cat,ps_car_12,ps_car_13,ps_car_15,ps_car_05_cat_0.5261814898236643,ps_car_07_cat_1.0,target
0,0.211296,1.918116,-0.805988,-0.587013,2.256576,1.607012,-0.370692,-0.735389,-5.015862e-01,-1.362990,-1.092421,-0.308070,0.342236,1.109956,0.274741,0
1,0.951489,-0.311952,-0.805988,1.703539,-0.443149,-1.213525,-0.370692,-0.039750,9.479614e-01,0.685453,2.886394,1.780666,-0.576081,-0.900937,0.274741,0
2,2.431874,-0.311952,-0.805988,-0.587013,-0.443149,1.324958,-0.370692,-1.778847,-3.552328e-16,-1.362990,0.347414,-0.218574,-0.091172,-0.900937,0.274741,0
3,-1.639185,-0.311952,-0.805988,-0.587013,2.256576,-1.495578,-0.370692,-0.039750,-5.745391e-01,1.709675,-1.092421,-0.564436,0.342236,1.109956,0.274741,0
4,-0.898992,-0.311952,-0.805988,1.703539,-0.443149,1.607012,-0.370692,-1.778847,-3.552328e-16,-1.362990,-1.092421,-0.503109,0.130960,-0.900937,0.274741,0


In [ ]:
# Salvando  a ABT para treinamento dos modelos - Treino e Teste
abt_01.to_csv(dir_base + 'abt_fs_rfe_train.csv')
abt_01_test.to_csv(dir_base + 'abt_fs_rfe_test.csv')